In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import torch

In [3]:
WEIGHTS_PATH = "/content/drive/MyDrive/nlp-inheritune-project/checkpoint_gpt2-medium-10layers_iter_15000.pt"

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model

In [5]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.9 MB/s eta 0:00:00


In [6]:
"""
nano-gpt implementation

Full definition of a GPT Language Model, all of it in this single file.
References:
1) the official GPT-2 TensorFlow implementation released by OpenAI:
https://github.com/openai/gpt-2/blob/master/src/model.py
2) huggingface/transformers PyTorch implementation:
https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py
"""

import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def crop_block_size(self, block_size):
        # model surgery to decrease the block size if necessary
        # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
        # but want to use a smaller block size for some smaller, simpler model
        assert block_size <= self.config.block_size
        self.config.block_size = block_size
        self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
        for block in self.transformer.h:
            if hasattr(block.attn, 'bias'):
                block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

    @classmethod
    def from_pretrained(cls, model_type, override_args=None):
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        override_args = override_args or {} # default to empty dict
        # only dropout can be overridden see more notes below
        assert all(k == 'dropout' for k in override_args)
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        config_args['bias'] = True # always True for GPT model checkpoints
        # we can override the dropout rate, if desired
        if 'dropout' in override_args:
            print(f"overriding dropout rate to {override_args['dropout']}")
            config_args['dropout'] = override_args['dropout']
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [118]:
model = GPT.from_pretrained("gpt2-medium")

loading weights from pretrained gpt: gpt2-medium
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 353.77M


In [8]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

# Load model

In [66]:
class MyGPT(GPT):
    def __init__(self, config, base_model):
        super().__init__(config)
        # Копируем первые 4 слоев из base_model
        self.transformer.h = nn.ModuleList([base_model.transformer.h[i] for i in range(10)])
        self.transformer.wte = base_model.transformer.wte
        self.transformer.wpe = base_model.transformer.wpe
        self.transformer.drop = base_model.transformer.drop
        self.transformer.ln_f = base_model.transformer.ln_f
        self.lm_head = base_model.lm_head


config = GPTConfig(
    block_size=1024,
    vocab_size=50257,
    n_layer=10,
    n_head=model.config.n_head,
    n_embd=model.config.n_embd,
    bias=True
)

# Модель с 4 слоями
my_model = MyGPT(config, model)

number of parameters: 177.43M


In [67]:
model.to(device)
my_model.to(device)

MyGPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-9): 10 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [70]:
# Generate
start_ids = encode("It is just a start")
x = (torch.tensor(start_ids, dtype=torch.long, device="cuda")[None, ...])

my_model.eval()
with torch.no_grad():
    y = my_model.generate(x, 200)
    print(decode(y[0].tolist()))
    print('---------------')

It is just a start paying attention to the underlying foundation principles behind these particular March 29th 2010 0000 00 00 00 00 ired, which unfortunately unfortunately failing miserably named. The only reason why airtight command warrants clausewitz et al signaling his own unique id quo quo ante based ˜²^\ . with respective ircettering network, which none whatsoever whatsoever (or indeedally...etc., etc.), which allows examinership over the course of multiple colored Mercedury parking vacations within the respective developments. And thusforth continuing onward, though as far tampering possible.
 intimidation : DEVICE systems based on the same kindologies such development-based relationships, which are divided vs through in order prior to:

toilette safety inductive  preferred acknowledgement Tiffany Tru Cannonneller, zewite-based approach CGI Cheerful Brownlee CIBUILDING TECH Developmentally oriented focused control allocation procreate.com addresses its own variety of expertise 

In [71]:
# load trained small model
checkpoint = torch.load(WEIGHTS_PATH, map_location=torch.device('cuda'))
my_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

# Generate

In [119]:
# Generate
model.to(device)

start_ids = encode("It is just a start")
x = (torch.tensor(start_ids, dtype=torch.long, device="cuda")[None, ...])

model.eval()
with torch.no_grad():
    y = model.generate(x, 200)
    print(decode(y[0].tolist()))
    print('---------------')

It is just a start, though. Meanwhile, Japan is evolving rapidly. Healthcare," she says.

Getting Japan's online habits moving<|endoftext|>Aakia is his bright-eyed messiah, kicking an almost-stuck incantation while discussing the resounding, universal declaration, "All people shouldn't worship pigs, jokers, horse shit, or the elderly."

About a minute later, Aakia unlocks her Quidditch kit in a neutral stance because she looks like she's secretly being Harry Potter on Canets captainduckravensofthest.com with just her knickers on, waves goodbye to teaspoons and defibber, and begins rummaging through her replaceable kit, becoming the youngest ever game caster in no time flat.

"Goodbye Daddy" by Robert Rector.<|endoftext|>Ron Paul

 outside the Governor's office in Madison, Wis., on Wed., June 17, 2011, in response to demand from conservative activists seeking him to turn in the names of his
---------------


# MMLU 5-shot evaluation

In [20]:
import os
import pandas as pd

In [21]:
# Install, and download MMLU
%pip install -e ../.

!curl -O https://people.eecs.berkeley.edu/~hendrycks/data.tar
!tar -xf data.tar
data_path = "data"

Obtaining file:///
ERROR: file:/// does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  158M  100  158M    0     0  79.6M      0  0:00:01  0:00:01 --:--:-- 79.6M


In [22]:

choices = ["A", "B", "C", "D"]
sys_msg = "The following are multiple choice questions (with answers) about {}."
def create_last_prompt(question, answers):
    user_prompt = f"{question}\n" + "\n".join([f"{choice}. {answer}" for choice, answer in zip(choices, answers)]) + "\nAnswer:"
    return user_prompt

def create_example_prompt(question, answers, right_answer):
  user_prompt = f"{question}\n" + "\n".join([f"{choice}. {answer}" for choice, answer in zip(choices, answers)]) + f"\nAnswer: {right_answer}"
  return user_prompt

def n_shot_prompt(sys_msg, user_prompts, last_prompt, subject):
  return sys_msg.format(subject)+ "\n" + "\n".join([prompt for prompt in user_prompts]) + "\n" +  last_prompt


In [23]:
questions_dir = "data/test"

In [24]:
one_subject_questions_path = os.listdir(questions_dir)[1]
one_subject_questions_path

'security_studies_test.csv'

In [25]:
def generate_few_shot_prompts(one_subject_questions_path, shot_number):
    subject = one_subject_questions_path.split('.')[0]
    df = pd.read_csv(os.path.join(questions_dir, one_subject_questions_path))
    num_prompts = len(df) // shot_number

    few_shot_prompts = []
    correct_answers = []
    for i in range(num_prompts):
        df_subset = df.iloc[i*shot_number:(i+1)*shot_number]


        user_prompts = [
            create_example_prompt(row[0], row[1:5], row[5])
            for _, row in df_subset.iterrows()
        ]


        last_prompt = create_last_prompt(df_subset.iloc[-1, 0], df_subset.iloc[-1, 1:5])
        few_shot_prompt = n_shot_prompt(sys_msg, user_prompts[:-1], last_prompt, subject)


        few_shot_prompts.append(few_shot_prompt)

        correct_answers.append(df_subset.iloc[-1, 5])

    return few_shot_prompts, correct_answers


In [26]:
prompts, correct_answers = generate_few_shot_prompts(one_subject_questions_path, 5)
prompts[0], correct_answers[0]

('The following are multiple choice questions (with answers) about security_studies_test.\nCan environmental changes be reconciled with national security interests?\nA. Environmental challenges to economic growth, such as natural resource management and employment, leads to diversification and research which may in the long-term result in technological advancements that bolster military programs.\nB. Environmental changes can undermine national security in many ways; including weakening the economic base that determines military capacity. So if the natural capital base of an economy erodes than so does the long-term capacity of its armed forces. Any developments however, will be \'human\' in impact.\nC. The environmental problems encountered by countries are principally internal. External threats remain within a country\'s control: external military threats will remain unchanged and a primary concern.\nD. All of these options.\nAnswer: B\nWhich of the following statements could be desc

In [123]:
import torch

# Предполагаем, что функции encode и decode уже определены
# а также модель и устройство (device) уже инициализированы

def calculate_accuracy(test_model, prompts, correct_answers):
    correct_count = 0

    for prompt, correct_answer in zip(prompts, correct_answers):
        # Encode the prompt
        start_ids = encode(prompt)
        x = torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...]

        test_model.eval()
        with torch.no_grad():
            # Generate the model's response
            y = test_model.generate(x, 1)
            generated_answer = decode(y[0].tolist())

            # Extract the actual answer from the model's response
            model_answer = generated_answer.strip().split()[-1]

            # Compare model's answer with the correct answer
            if model_answer == correct_answer:
                correct_count += 1

    accuracy = correct_count / len(prompts)
    return accuracy




In [124]:
def count_acc_all_subjects(test_model, data_dir_path, num_shots):
  data_files = os.listdir(data_dir_path)
  accuracy_dict = {}
  for file in data_files:
    prompts, correct_answers = generate_few_shot_prompts(file, num_shots)
    accuracy = calculate_accuracy(test_model, prompts, correct_answers)
    accuracy_dict[file] = accuracy
    print(f"Calculated for {file}, accuracy: {accuracy}")
  return accuracy_dict

## Try 1

In [62]:
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [144]:
start_ids = encode(prompts[0])
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

model.eval()
with torch.no_grad():
    y = model.generate(x, 1)
    print(decode(y[0].tolist()))

The following are multiple choice questions (with answers) about security_studies_test.
Can environmental changes be reconciled with national security interests?
A. Environmental challenges to economic growth, such as natural resource management and employment, leads to diversification and research which may in the long-term result in technological advancements that bolster military programs.
B. Environmental changes can undermine national security in many ways; including weakening the economic base that determines military capacity. So if the natural capital base of an economy erodes than so does the long-term capacity of its armed forces. Any developments however, will be 'human' in impact.
C. The environmental problems encountered by countries are principally internal. External threats remain within a country's control: external military threats will remain unchanged and a primary concern.
D. All of these options.
Answer: B
Which of the following statements could be described as a l

# Evaluate gpt-2-medium

In [115]:
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [148]:
accuracy_dict = count_acc_all_subjects(model, questions_dir, 5)
mean_accuracy = sum(accuracy_dict.values()) / len(accuracy_dict)
mean_accuracy

Calculated for computer_security_test.csv, accuracy: 0.21052631578947367
Calculated for security_studies_test.csv, accuracy: 0.25
Calculated for professional_medicine_test.csv, accuracy: 0.2777777777777778
Calculated for high_school_biology_test.csv, accuracy: 0.26229508196721313
Calculated for business_ethics_test.csv, accuracy: 0.21052631578947367
Calculated for nutrition_test.csv, accuracy: 0.2459016393442623
Calculated for professional_psychology_test.csv, accuracy: 0.20491803278688525
Calculated for high_school_statistics_test.csv, accuracy: 0.3488372093023256
Calculated for medical_genetics_test.csv, accuracy: 0.21052631578947367
Calculated for sociology_test.csv, accuracy: 0.225
Calculated for high_school_physics_test.csv, accuracy: 0.16666666666666666
Calculated for college_biology_test.csv, accuracy: 0.32142857142857145
Calculated for high_school_government_and_politics_test.csv, accuracy: 0.18421052631578946
Calculated for jurisprudence_test.csv, accuracy: 0.23809523809523808

0.21789978347983258

In [ ]:
import json

with open('gpt-2-medium_mmlu_5_shot_v1.json', 'w') as f:
    json.dump(accuracy_dict, f)

# Evaluation inherited gpt-2-medium without training, just initiated with transformer blocks

In [ ]:
my_model.to(device)

MyGPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
accuracy_dict = count_acc_all_subjects(my_model, questions_dir, 5)
mean_accuracy = sum(accuracy_dict.values()) / len(accuracy_dict)
mean_accuracy

Calculated for management_test.csv, accuracy: 0.0
Calculated for machine_learning_test.csv, accuracy: 0.045454545454545456
Calculated for high_school_psychology_test.csv, accuracy: 0.0
Calculated for abstract_algebra_test.csv, accuracy: 0.0
Calculated for high_school_computer_science_test.csv, accuracy: 0.05263157894736842
Calculated for formal_logic_test.csv, accuracy: 0.0
Calculated for virology_test.csv, accuracy: 0.0
Calculated for professional_psychology_test.csv, accuracy: 0.00819672131147541
Calculated for high_school_chemistry_test.csv, accuracy: 0.0
Calculated for high_school_european_history_test.csv, accuracy: 0.03125
Calculated for global_facts_test.csv, accuracy: 0.0
Calculated for moral_scenarios_test.csv, accuracy: 0.016853932584269662
Calculated for nutrition_test.csv, accuracy: 0.01639344262295082
Calculated for high_school_microeconomics_test.csv, accuracy: 0.02127659574468085
Calculated for miscellaneous_test.csv, accuracy: 0.0
Calculated for clinical_knowledge_test.

0.0066310333022282725

In [ ]:
import json

with open('inherited_only_init_gpt2-medium_mmlu_5_shot.json', 'w') as f:
    json.dump(accuracy_dict, f)

# Evaluation inherited gpt-2-medium 20000 iterations

In [ ]:
my_model.to(device)

MyGPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
accuracy_dict = count_acc_all_subjects(my_model, questions_dir, 5)
mean_accuracy = sum(accuracy_dict.values()) / len(accuracy_dict)
mean_accuracy

Calculated for computer_security_test.csv, accuracy: 0.05263157894736842
Calculated for security_studies_test.csv, accuracy: 0.0
Calculated for professional_medicine_test.csv, accuracy: 0.0
Calculated for high_school_biology_test.csv, accuracy: 0.01639344262295082
Calculated for business_ethics_test.csv, accuracy: 0.0
Calculated for nutrition_test.csv, accuracy: 0.04918032786885246
Calculated for professional_psychology_test.csv, accuracy: 0.01639344262295082
Calculated for high_school_statistics_test.csv, accuracy: 0.046511627906976744
Calculated for medical_genetics_test.csv, accuracy: 0.10526315789473684
Calculated for sociology_test.csv, accuracy: 0.0
Calculated for high_school_physics_test.csv, accuracy: 0.0
Calculated for college_biology_test.csv, accuracy: 0.07142857142857142
Calculated for high_school_government_and_politics_test.csv, accuracy: 0.0
Calculated for jurisprudence_test.csv, accuracy: 0.0
Calculated for high_school_world_history_test.csv, accuracy: 0.021276595744680

0.02714187681961806

In [ ]:
import json

with open('inherited_20000_gpt2-medium_mmlu_5_shot.json', 'w') as f:
    json.dump(accuracy_dict, f)

# Evaluation inherited gpt-2-medium 10 layers 15000 iterations

In [75]:
my_model.to(device)

MyGPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-9): 10 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [76]:
accuracy_dict = count_acc_all_subjects(my_model, questions_dir, 5)
mean_accuracy = sum(accuracy_dict.values()) / len(accuracy_dict)
mean_accuracy

Calculated for computer_security_test.csv, accuracy: 0.0
Calculated for security_studies_test.csv, accuracy: 0.020833333333333332
Calculated for professional_medicine_test.csv, accuracy: 0.0
Calculated for high_school_biology_test.csv, accuracy: 0.01639344262295082
Calculated for business_ethics_test.csv, accuracy: 0.0
Calculated for nutrition_test.csv, accuracy: 0.01639344262295082
Calculated for professional_psychology_test.csv, accuracy: 0.0
Calculated for high_school_statistics_test.csv, accuracy: 0.0
Calculated for medical_genetics_test.csv, accuracy: 0.0
Calculated for sociology_test.csv, accuracy: 0.0
Calculated for high_school_physics_test.csv, accuracy: 0.0
Calculated for college_biology_test.csv, accuracy: 0.14285714285714285
Calculated for high_school_government_and_politics_test.csv, accuracy: 0.0
Calculated for jurisprudence_test.csv, accuracy: 0.0
Calculated for high_school_world_history_test.csv, accuracy: 0.0
Calculated for high_school_european_history_test.csv, accurac

0.02885811785358698

In [128]:
accuracy_dict = count_acc_all_subjects(my_model, questions_dir, 5)
mean_accuracy = sum(accuracy_dict.values()) / len(accuracy_dict)
mean_accuracy

Calculated for computer_security_test.csv, accuracy: 0.0
Calculated for security_studies_test.csv, accuracy: 0.0
Calculated for professional_medicine_test.csv, accuracy: 0.0
Calculated for high_school_biology_test.csv, accuracy: 0.0
Calculated for business_ethics_test.csv, accuracy: 0.0
Calculated for nutrition_test.csv, accuracy: 0.0
Calculated for professional_psychology_test.csv, accuracy: 0.01639344262295082
Calculated for high_school_statistics_test.csv, accuracy: 0.0
Calculated for medical_genetics_test.csv, accuracy: 0.15789473684210525
Calculated for sociology_test.csv, accuracy: 0.0
Calculated for high_school_physics_test.csv, accuracy: 0.0
Calculated for college_biology_test.csv, accuracy: 0.10714285714285714
Calculated for high_school_government_and_politics_test.csv, accuracy: 0.0
Calculated for jurisprudence_test.csv, accuracy: 0.0
Calculated for high_school_world_history_test.csv, accuracy: 0.0
Calculated for high_school_european_history_test.csv, accuracy: 0.0
Calculated

0.04233096080837395

In [77]:
import json

with open('inherited_15000_gpt2-medium-10layers_mmlu_5_shot.json', 'w') as f:
    json.dump(accuracy_dict, f)